# Instrucciones para correr el programa

1) Carga de base de datos de libros:
  * **Scrap**: Para obtener los libros usando el Scrap, se debe acceder a dicha seccion y ejecutar el codigo, esto traera los libros, generara un df y lo guardara en formato csv.
  * **Carga de archivo csv**: Cargar el archivo csv correspondiente al entorno de ejecucion, si se ejecuto anteriormente el Scrap, este archivo ya estara en el directorio.

2) Declarar todas las funciones necesarias en las secciones **Procesamiento de texto**, **Buscar libros por reseña**, **Buscar libros por genero y autor**.

3) Ejecutar el **Cliente**.

# Instalacion e importacion las librerias necesarias

In [ ]:
!pip install sentence-transformers
!pip install fuzzywuzzy
!pip install nltk
!pip install python-Levenshtein

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import random
from fuzzywuzzy import process
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from gensim.models import Word2Vec
import unicodedata

In [130]:
from sentence_transformers import SentenceTransformer, util
# Función para obtener embeddings de oraciones
def get_sentence_embeddings(sentences):
    model = SentenceTransformer('msmarco-MiniLM-L-6-v3')
    embeddings = model.encode(sentences)
    return embeddings

In [133]:
generos = ["ficcion", "historia", "misterio", "infantil", "arte", "economia", "humor", "poesia", "religion", "terror", "aventuras", "ciencia", "cuentos", "politica", "teatro", "tecnologia", "novela", "humor", "musica", "drama", "fantasia", "economia", "ciencia-ficcion"]
print(len(generos))

23


# Scrap

Realizamos el scrap a la pagina lectulandia con el objetivo de llenar nuestra base de datos con libros.
Para esta aplicacion usamos la libreria BeautifulSoup4 ya que la estructura de la pagina no presentaba una gran dificultad y con las herramientas de dicha libreria era suficiente para encontrar y recopilar todo lo necesario.

En primer lugar armamos las url correspondientes, definimos un objeto data con las claves Genero, Titulo, Autor y Descripcion que luego vamos a llenar con los datos correspondientes a cada libro.
Tambien definimos una lista con todos los generos que deseamos incluir en nuestra base de datos de libros.

Finalmente, iterando sobre la lista de generos, por cada genero ingresamos a la url de cada libro disponible, esto lo hicimos debido a que, extrayendo la informacion del libro desde la url del genero, la descripcion estaba incompleta, por lo que decidimos ingresar a cada libro para poder extraer todos sus datos completos.

Finalmente convertimos el objeto data a un df de pandas, generamos los Embeddings para la descripcion de cada libro y lo almacenamos en una nueva columna llamada Embeddings y lo guardamos en un archivo csv.

In [134]:
# Obtener el contenido HTML de la página principal
url_index = 'https://ww3.lectulandia.com'
response_index = requests.get(url_index)
html_content_index = response_index.text
soup_index = BeautifulSoup(html_content_index, 'html.parser')

# Armamos un objeto vacio donde vamos a ir almacenando los libros
data = {'Genero': [],
        'Titulo': [],
        'Autor': [],
        'Descripcion': []}

# generos = ["ficcion", "historia", "misterio", "infantil", "arte", "economia", "humor", "poesia", "religion", "terror", "aventuras", "ciencia", "cuentos", "politica", "teatro", "tecnologia"]
generos = ["ficcion", "historia", "misterio", "infantil", "arte", "economia", "humor", "poesia", "religion", "terror", "aventuras", "ciencia", "cuentos", "politica", "teatro", "tecnologia", "novela", "humor", "musica", "drama", "fantasia", "economia", "ciencia-ficcion"]

# Encontramos el tag <section> con id 'secgenero' que es el que contiene a todos los generos
soup_generos = soup_index.find('section', id='secgenero')

# Iteramos la lista de generos, y obetenemos el enlace correspondiente
for genero in generos:
  # Buscamos el tag <a> que en e href contiene la url correspondiente
  soup = soup_generos.find('a', href=f'/genero/{genero}/')
  count_genero = 0
  if soup:
    # Construir el enlace completo y armamos un nuevo objeto soup
    url_genero = url_index + soup["href"]
    response_genero = requests.get(url_genero)
    html_content_genero = response_genero.text
    soup_genero = BeautifulSoup(html_content_genero, 'html.parser')
    # Encuentra todas las etiquetas <article> con la clase "card"
    lista_articulos = soup_genero.find_all('article', class_='card')
    for articulo in lista_articulos:
      if count_genero == 24:
        break
      else:
        enlace_card = articulo.find('a', class_='card-click-target')
        href_value = enlace_card.get('href')
        url_libro = url_index + href_value
        response_libro = requests.get(url_libro)
        html_content_libro = response_libro.text
        soup_libro = BeautifulSoup(html_content_libro, 'html.parser')

        titulo = soup_libro.find('div', id='title').find('h1').text.strip()
        if titulo in data['Titulo']:
          continue
        else:
          data['Titulo'].append(titulo)
          data['Genero'].append(genero)
          autor = soup_libro.find('div', id='autor').find('a').text.strip()
          data['Autor'].append(autor)
          desc = soup_libro.find('div', id='sinopsis').get_text(strip=True)
          data['Descripcion'].append(desc)
          count_genero += 1

df = pd.DataFrame(data)
df['Embeddings'] = df['Descripcion'].apply(lambda x: get_sentence_embeddings([x])[0])
df.to_csv('df_libros.csv', index=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# Importacion de archivo csv

Definimos una funcion con el objetivo de convertir un str que contiene vectores a un objeto de np.array, esto debido a que, al cargar el dataset desde un archivo csv, el tipo de dato de la columna Embeddings es str y necesitamos que sea np.array para las aplicaciones del programa



In [135]:
import numpy as np
def cadena_a_arreglo_np(cadena):
    cadena_limpia = cadena.replace("[", "").replace("]", "").replace("\n", "")
    elementos = cadena_limpia.split()
    elementos_numeros = [float(elem) for elem in elementos]
    arreglo_np = np.array(elementos_numeros)
    return arreglo_np

Cargamos el df desde un archivo csv y convertimos la columan Embeddings a tipo de dato np.array con la funcion previamente declarada.

In [136]:
df_libros = pd.read_csv('df_libros.csv')
# Aplicar la función a toda la columna "Embeddings"
df_libros['Embeddings'] = df_libros['Embeddings'].apply(cadena_a_arreglo_np)

In [ ]:
df_libros

# Procesamiento de texto

En esta seccion de  codigo definimos funciones utiles que usaremos proximamente.

* **preprocess_text**: Es una funcion que recibe una cadena de texto y le aplica un procesamiento para dejarla lo mas limpia posible, unicamente con palabras clave, pasando el texto a minuscula, eliminando caracteres especiales, aplicando una tokenizacion con word_tokenize y filtrando palabras comunes, conectotores, etc. con stop_words de nltk.

* **tokenizar_con_bert**: Funcion para tokenizar una cadena de texto con Bert Sentence Tokenizer, utilizamos este metodo porque nos parecio el mas completo y adecuado a la hora de tokenizar un fragmenteo de texto largo o un documento.

* **Levenshtein_distance**: Funcion que recibe una palabra y una lista de palabras y devuelve la palabra de la lista que tiene la mayor similitud a la palabra ingresada usando la distancia de Levenshtein.

* **encontrar_genero_similar**: Esta funcion la implementamos para pasar una frase o cadena de texto y detectar si dentro de ella hay una palabra que se asemeje a alguna de la lista de palabras pasada, usando la funcion previamente mencionada Levenshtein_distance.

In [119]:
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\sáéíóúüñ]', '', text)
    # Tokenización
    tokens = word_tokenize(text, language='spanish')
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Reconstruir el texto preprocesado
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

# Función para tokenizar el texto utilizando BERT (para las descripciones)
def tokenizar_con_bert(texto):
  # Cargar el tokenizador BERT pre-entrenado en español
    tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
    return tokenizer.tokenize(texto)

# Función para tokenizar las sinopsis
def tokenizar_con_wordtokenizer(texto):
    return word_tokenize(texto.lower())

# Funcion para tokenizar palabras (para genero y autor)
def tokenizar_palabras(texto):
    model = BertModel.from_pretrained('bert-base-multilingual-cased')
    return model.tokenize(texto)

import Levenshtein
def Levenshtein_distance(palabra, lista_palabras):
    mejor_similitud = float('inf')
    palabra_similar = None
    for palabra_lista in lista_palabras:
        distancia = Levenshtein.distance(palabra, palabra_lista)
        if distancia < mejor_similitud:
            mejor_similitud = distancia
            palabra_similar = palabra_lista
    return palabra_similar

def encontrar_genero_similar(frase, lista_generos):
    palabras = frase.split()
    genero_similar = None
    mejor_distancia = float('inf')
    for palabra in palabras:
        palabra_similar = Levenshtein_distance(palabra, lista_generos)
        distancia = Levenshtein.distance(palabra, palabra_similar)
        if distancia < mejor_distancia:
            mejor_distancia = distancia
            genero_similar = palabra_similar
    return genero_similar

# Buscar libros por **reseña**

Definimos la funcion **similarity_descripcion** a la que le pasamos una cadena de texto, correspondiente a la solicitud del usuario, pasamos un df donde se encuentran todos nuestros libros y la cantidad que deseamos traer, por defecto son 3.

Lo primero que hacemos es limpiar la entrada del usuario con la funcion **preprocess_text**.
Traemos la lista de generos presentes en el df, a que genero se asimila la entrada de usuario usando la funcion **encontrar_genero_similar** y generamos un df con dicho genero.
Generamos los Embeddings para la entrada del usuario usando la misma funcion usada para generar los embeddings de las descripciones de cada libro, buscamos las similitudes entre los embeddings usando la **distancia del coseno** y retornamos el top 3.

In [120]:
# Función para calcular similitud entre una consulta y descripciones
def similarity_descripcion(user_query, df, top_n=3):
    preprocessed_query = preprocess_text(user_query)
    generos = df_libros['Genero'].unique()
    genero_similar = encontrar_genero_similar(user_query, generos)
    libros_genero = df[df['Genero'] == genero_similar]
    query_embedding = get_sentence_embeddings([preprocessed_query])[0]
    similarities = libros_genero['Embeddings'].apply(lambda x: util.cos_sim(query_embedding.astype(np.float64), x).item())
    top_indices = similarities.nlargest(top_n).index
    return df.iloc[top_indices][['Titulo', 'Autor', 'Genero', 'Descripcion']]

# Buscar libros por **Genero** y **Autor**

Para la buqueda por genero y autor implementamos la misma logica, principalmente usando la funcion **Levenshtein_distance**.

Las funciones **similarity_genero** y **similarity_autor** basicamente lo que hacen es tomar una entrada del usuario, un df para obtener todos los generos presentes en el mismo y retornar cual es el genero correspondiente o que mas se asimila al requerido por el usuario.

Con las funciones **recomendar_libros_genero** y **recomendar_libros_autor** implementamos las funciones antes mencionadas para obtener el genero o autor correspondiente, filtramos y retornamos el df con los libros correspondientes a ese genero o autor.

In [121]:
# Función para calcular la palabra más cercana a la escrita y recomendar 3 libros de ese género
def similarity_genero(user_query, df, top=3):
    generos = df_libros['Genero'].unique()
    genero_similar = Levenshtein_distance(user_query, generos)
    return genero_similar

def recomendar_libros_genero(user_genero, df, top=3):
    generos = df_libros['Genero'].unique()
    genero_similar = similarity_genero(user_genero, generos)
    libros_recomendados_genero = df_libros[df_libros['Genero'] == genero_similar].head(top)
    return libros_recomendados_genero

In [122]:
# Función para calcular la palabra más cercana a la escrita y recomendar 3 libros de ese autor
def similarity_autor(user_query, df, top=3):
    autores = df_libros['Autor'].unique()
    autor_similar = Levenshtein_distance(user_query, autores)
    return autor_similar

def recomendar_libros_autor(user_autor, df, top=3):
    autores = df_libros['Autor'].unique()
    autor_similar = similarity_autor(user_autor, autores)
    libros_recomendados_autor = df_libros[df_libros['Autor'] == autor_similar].head(top)
    libros_recomendados_autor.drop('Embeddings', axis=1, inplace=True)
    return libros_recomendados_autor

# Cliente

In [126]:
from tabulate import tabulate

# Solicitar al usuario cómo desea realizar la búsqueda
search_type = input("¿Cómo deseas buscar? (reseña/autor/género): ").strip().lower()

if search_type == "reseña" or search_type == "resenia":
    user_query = input("Ingresa tu consulta por reseña: ")
    recommended_books = similarity_descripcion(str(user_query), df_libros)
elif search_type == "autor":
    author_name = input("Ingresa el nombre del autor: ").strip()
    recommended_books = recomendar_libros_autor(author_name, df_libros)
elif search_type == "género" or search_type == "genero":
    genre = input("Ingresa el género literario: ").strip()
    recommended_books = recomendar_libros_genero(genre, df_libros)
else:
    recommended_books = "Opción de búsqueda no válida."

# Mostrar resultados en una tabla
if isinstance(recommended_books, pd.DataFrame):
    print(tabulate(recommended_books.drop(columns=['Embeddings', 'similarity'], errors='ignore'), headers='keys', tablefmt='pretty'))
else:
    print(recommended_books)

¿Cómo deseas buscar? (reseña/autor/género): autor
Ingresa el nombre del autor: Franz
+---+---------+----------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   | Genero  |     Titulo     |    Autor    |                                                                                                                                      

# Conclusiones finales

### Scrap

Consideramos que el scrap realizado funciona bien y cumple con lo requerido, sin embargo es muy mejorable, principalmente en la fuente y la forma en la que recopila y trae los libros. Nosotros lo hicimos ingresando por genero y le asigna un genero a cada uno, por ahi una mejor opcion era agarrar libros desde el index y almacenar en la columna Genero un array con sus generos ya que apreciamos en la pagina que por lo general cada libro tiene mas de un genero, tambien se podria implementar una funcion donde se ingrese unicamente la url y la cantidad de libros requerida, como asi muchas mejoras mas, por cuestiones de tiempo optamos por dejarlo asi ya que funciona bien y cumple con el objetivo.

### Obtener libros por **Reseña**

Al principio experimentamos problemas debido a que nuestra basde de datos de libros era muy acotada, lo que hacia que a la hora de buscar, no encontrara las coincidencias mas acordes, por ejemplo, a la hora de buscar "libros de terror" generalmente encontraba 1 o 2 libros acordes y uno que nada tenia que ver con lo solicitado.

Decidimos implementar un poco de los metodos utilizados en la busqueda por genero y detectamos dentro de la entrada del usuario, alguna palabra con cercania con alguno de los los generos presentes en nuestra base de datos, esto nos facilita nos facilita la posterior busqueda por similitud acotando la lista de libros.
Finalmente calculamos la distancia entre los embeddings de la entrada y los embeddings de la descripcion de cada libro y obtenemos los 3 primeros.

De esta forma logramos obtener una respuesta mas cercana a lo solicitado, sin embargo, es alta la posibilidad de detectar una palabra incorrecta a la hora de ecnontrar un genero y hacer que falle, tambien, otra falencia podria ser que se encuentra ligado directamente a una busqueda interna de genero.

